In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import cvlib as cv
import pandas as pd
from cvlib.object_detection import draw_bbox
from ipywidgets import *

Using TensorFlow backend.


In [17]:
class ImageFile(object):
    """A file that is a picture of droplets

    Attributes:
        path: The location of the file
        name: the file name
    """

    def __init__(self, path, name):
        """Returns an ImageFile object"""
        self.path = path
        self.name = name

    def __repr__(self):
         return "Test()"
    def __str__(self):
         return str(vars(self))    



class DropletImage(object):
    """An image of a single droplet
    Attributes:
        original_image: the original image the droplet is from (path+name)
        path: the path to the  droplet image
        id_number: the id of the droplet
        position: the position of the droplet in the original image
        radius: the radius of the droplet
        type: the type of droplet
    """

    def __init__(self, original_image_path=None, original_image_name=None,path=None,
                 id_number=None, position=(None, None), radius=None, type=None):
        """Returns an DropletImage object"""
        self.original_image_path = original_image_path
        self.original_image_name = original_image_name
        self.path = path
        self.id = id_number
        self.position = position
        self.radius = radius
        self.type = radius

#    def set_position(self, position=(None, None)):
#        """Set the position of the droplet in the original image"""
#        self.position = position

    def gen_descriptor(self):
        end

In [18]:
test_image_1 = ImageFile(
    '/Users/olofjonsson/Projects/Ice/Droplet_codes/Preprocessing/Test_Images',
    '1.5mm_53.6ul_254.5kHz.png')

print(test_image_1)

test_droplet1=DropletImage(test_image_1.path,test_image_1.name)

{'path': '/Users/olofjonsson/Projects/Ice/Droplet_codes/Preprocessing/Test_Images', 'name': '1.5mm_53.6ul_254.5kHz.png'}
